# Чат-бот: погода
  
### Найти бота в Телеграм можно по [ссылке](https://t.me/Garganova_weather_bot)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import nltk
from nltk.corpus import stopwords

import pymorphy2
import collections 
import multiprocessing

import telebot

from bs4 import BeautifulSoup
import requests as rq
import re

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Наш бот будет уметь различать тон сообщения, и реагировать на плохие сообщения и на хорошие по разному. Для этого необходимо провести анализ тональности текста.

### Анализ тональности текста (sentiment analysis)

Для обучения возьмем [корпус коротких текстов Юлии Рубцовой](http://study.mokoron.com), сформированный на основе русскоязычных сообщений из Twitter.

In [ ]:
# Считываем данные

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('/Users/elizaveta/Downloads/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('/Users/elizaveta/Downloads/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [ ]:
# Удаляем ненужные символы

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

In [ ]:
data = [preprocess_text(t) for t in raw_data]
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

**Нормализуем слова**

In [ ]:
#Функция нормализации

morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [ ]:
#Распараллелим (Все равно работает долго!)

pool = multiprocessing.Pool()
norm_x_train = pool.map(lemmatize, x_train)

In [ ]:
# Скачиваем стоп-слова, чтобы удалить их из рассмотрения

nltk.download('stopwords')

In [ ]:
# Удаляем стоп-слова

stopwords_set = set(stopwords.words("russian"))
# А еще стоит удалить некоторые другие бессмысленные слова, встречающиеся очень часто
stopwords_set.add('user')
stopwords_set.add('url')
stopwords_set.add('rt')
stopwords_set.add('весь')
stopwords_set.add('это')
stopwords_set.add('d')

for i in range(len(norm_x_train)):
    norm_x_train[i] = list(set(norm_x_train[i]) - stopwords_set)   

**Векторизация TF-IDF**

In [ ]:
# Соединим нормализованные слова обратно в тексты

train_texts = []
for i in range(len(norm_x_train)):
    train_texts.append(' '.join(norm_x_train[i]))

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_texts)

**Обучаем Логистическую регрессию**

In [ ]:
logreg_clf = LogisticRegression()
logreg_clf.fit(X_train, y_train)

**Проверка точности предсказаний**

In [ ]:
# Соединим теперь всю обработку в одну функцию, чтобы также векторизовать тестовую выборку

def preproc_x(x_test):
    norm_x_test = pool.map(lemmatize, x_test)
    for i in range(len(norm_x_test)):
        norm_x_test[i] = list(set(norm_x_test[i]) - stopwords_set) 
    test_texts = []
    for i in range(len(norm_x_test)):
        test_texts.append(' '.join(norm_x_test[i]))
    X_test = vectorizer.transform(test_texts)
    return X_test

In [ ]:
# Предсказания

X_test = preproc_x(x_test)
y_pred = logreg_clf.predict(X_test)

In [ ]:
# Вычисляем метрики

print('Accuracy:',  accuracy_score(y_test, y_pred))       
print('Precision:',  precision_score(y_test, y_pred))
print('Recall:',  recall_score(y_test, y_pred))
print('F1:',  f1_score(y_test, y_pred))

### Парсим погоду
Гисметео и Яндекс.Погода весьма не сговорчивы для парсинга, пришлось довольствоваться Рамблер погодой.

In [ ]:
# msc - Москва, spb - СПб

url_msc = 'https://weather.rambler.ru/v-moskve/3-days/'
url_spb = 'https://weather.rambler.ru/v-sankt-peterburge/3-days/'

html_msc = rq.get(url_msc).text
html_spb = rq.get(url_spb).text

soup_msc = BeautifulSoup(html_msc,'html.parser')
soup_spb = BeautifulSoup(html_spb,'html.parser')

In [ ]:
# day - число месяца, temp - температура, wind - ветер, wet - вероятность осадков

day_msc = [soup_msc.findAll("span", {"class":'_30Ou'})[i].text for i in range(3)]
temp_msc = [soup_msc.findAll("span", {"class":'_3tD7'})[i].text for i in range(3)]
wind_msc = [soup_msc.findAll("span", {"class":'_3ty-'})[i].text for i in range(3)]
wet_msc = [soup_msc.findAll("span", {"class":'_3TiQ'})[i].text for i in range(3)]

day_spb = [soup_spb.findAll("span", {"class":'_30Ou'})[i].text for i in range(3)]
temp_spb = [soup_spb.findAll("span", {"class":'_3tD7'})[i].text for i in range(3)]
wind_spb = [soup_spb.findAll("span", {"class":'_3ty-'})[i].text for i in range(3)]
wet_spb = [soup_spb.findAll("span", {"class":'_3TiQ'})[i].text for i in range(3)]

In [ ]:
# Добавим еще привычные значки с осадками, зависящие от вероятности осадков

def wet_pict(wet_list):
    pictures = []
    integers = []  # это будет список со значениями вероятностей в цифрах (без знака процента)
    for i in range(len(wet_list)):
        integers.append(int(re.sub('%', '', wet_list[i])))
        if integers[-1] == 0:
            pictures.append('🌤')  # может быть и солнечно, а могут быть и облака при нулевой вероятности осадков
        elif integers[-1] >= 50:
            pictures.append('🌧')
        else:
            pictures.append('☁️')
    return pictures

In [ ]:
wet_pict_msc = wet_pict(wet_msc)
wet_pict_spb = wet_pict(wet_spb)

In [ ]:
wet_pict_msc

### А теперь приступим к созданию самого бота

In [ ]:
bot = telebot.TeleBot('1401865395:AAEwGnbN8K69WutpTuorWN-r3R8BtHmxxXo')

In [ ]:
# Кнопки для выбора города

cities = telebot.types.InlineKeyboardMarkup()
cities.add(telebot.types.InlineKeyboardButton(text="Москва", callback_data="msc"))
cities.add(telebot.types.InlineKeyboardButton(text="Санкт-Петербург", callback_data="spb"))

In [ ]:
# Обработка сообщений

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    
    # Анализ тональности сообщения
    mes = logreg_clf.predict(preproc_x([message.text]))[0]
    
    # Типовые вопросы
    t = re.search(r'(проща)|(до свидания)|(до скоро)|(уйти)|(заверш)|(закр)|(выйти)|(отключ)', message.text.lower()) is None
    w = re.search(r'(погод)|(спб)|(питер)|(санкт)|(петер)|(мск)|(москв)', message.text.lower()) is None
    
    # Приветствие
    if (message.text == "/start") or (message.text.lower() == "привет"):
        bot.send_message(message.from_user.id,
                         "Привет! Я предсказываю погоду ☀️ Для какого города ты бы хотел ее узнать?", reply_markup=cities)
    # Прощание
    elif message.text.lower() == "пока":
        bot.send_message(message.from_user.id, 'До скорой встречи! 👋🏻')
    elif t == False:
        bot.send_message(message.from_user.id, 'Хочешь попрощаться? Напиши мне "Пока"')
    
    # Погода в другом городе?
    elif w == False:
        bot.send_message(message.from_user.id, 'Хочешь теперь узнать погоду в другом городе? Выбирай!', reply_markup=cities)
    
    # Положительное или отрицательное сообщение?
    elif mes == 1:
        bot.send_message(message.from_user.id, "Чувствую, что ты написал мне что-то хорошее, но что - понять не могу 🧐 К сожалению, я умею только предсказывать погоду. Хочешь еще раз скажу? Выбирай город!", reply_markup=cities)
    else:
        bot.send_message(message.from_user.id, "Чувствую, что ты написал мне что-то плохое, но за что? Я ведь просто бот 😔 Если я плохо работаю, можешь пожаловаться моей создательнице: @lizagarganova")
        bot.send_sticker(message.from_user.id, "CAACAgIAAxkBAAKCb1_YcC8lMuLIC65QXnbkJlvPYCVmAAJiBQACP5XMCkCIeReG4EyMHgQ")
        bot.send_message(message.from_user.id, "Могу еще чем-то помочь?")

In [ ]:
# Ответ на выбор города

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "msc":
        msg = 'В Москве ' + day_msc[0] + ' ' + wet_pict_msc[0] + '\nТемпература воздуха ' + temp_msc[0] + '\nВетер ' + \
            wind_msc[0] + '\nВероятность осадков ' + wet_msc[0] + \
            '\n\n' + day_msc[1] + ' ' + wet_pict_msc[1] + '\nТемпература воздуха ' + temp_msc[1] + '\nВетер ' + \
            wind_msc[1] + '\nВероятность осадков ' + wet_msc[1] + \
            '\n\n' + day_msc[2] + ' ' + wet_pict_msc[2] + '\nТемпература воздуха ' + temp_msc[2] + '\nВетер ' + \
            wind_msc[2] + '\nВероятность осадков ' + wet_msc[2]
        bot.send_message(call.message.chat.id, msg)
        bot.send_message(call.message.chat.id, 'Могу ли быть полезен чем-то еще?')
    elif call.data == "spb":
        msg = 'В Санкт-Петербурге ' + day_spb[0] + ' ' + wet_pict_spb[0] + '\nТемпература воздуха ' + temp_spb[0] + '\nВетер ' + \
            wind_spb[0] + '\nВероятность осадков ' + wet_spb[0] + \
            '\n\n' + day_spb[1] + ' ' + wet_pict_spb[1] + '\nТемпература воздуха ' + temp_spb[1] + '\nВетер ' + \
            wind_spb[1] + '\nВероятность осадков ' + wet_spb[1] + \
            '\n\n' + day_spb[2] + ' ' + wet_pict_spb[2] + '\nТемпература воздуха ' + temp_spb[2] + '\nВетер ' + \
            wind_spb[2] + '\nВероятность осадков ' + wet_spb[2] 
        bot.send_message(call.message.chat.id, msg)
        bot.send_message(call.message.chat.id, 'Могу ли быть полезен чем-то еще?')

In [ ]:
# Запускаем нашего бота

bot.polling(none_stop=True, interval=0)